# QT User Meeting Tutorial 1 - Demo 1
## Qubit spectroscopy of a finFET

Copyright (C) 2022 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

# 0. General Imports

In [ ]:
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
import numpy as np
import time
from random import random

from qccs.simple import *
from qccs.pulse_sheet_viewer.pulse_sheet_viewer import show_pulse_sheet


# 1. Device Setup and Calibration

## 1.1 Define device setup

In [ ]:
descriptor=f"""\
instrument_list:
  HDAWG:
  - address: DEV8297
    uid: device_hdawg 
connections:
  device_hdawg:    
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/0, SIGOUTS/1]
    - rf_signal: q0/coulomb_line_1
      ports: [SIGOUTS/2]        
    - rf_signal: q0/coulomb_line_2
      ports: [SIGOUTS/3]
"""

## 1.2 Calibration

In [ ]:
def calibrate_devices(device_setup):
    
    device_setup.logical_signal_groups["q0"].logical_signals["drive_line"].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q0_osc",
            frequency=123e6,
            modulation_type=ModulationType.HARDWARE),
    )

## 1.3 Create device setup

In [ ]:
device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host='localhost',
    server_port=8004,
    setup_name='MySetup',
) 
calibrate_devices(device_setup)

# 2. Pulsed Qubit Spectroscopy

## 2.1 Define pulses

In [ ]:
## constant definition
len_cycle_coulomb = 400e-9

# define three stages of gate pulses
coulomb_reset = pulse_library.const(
    uid="coulomb_reset", length=len_cycle_coulomb/2, amplitude=0.5
    )
coulomb_manipulate = pulse_library.const(
    uid="coulomb_manipulate", length=len_cycle_coulomb/2, amplitude=0.75
    )

# define drive pulse
drive_pulse = pulse_library.gaussian(
    uid="drive_pulse", length=len_cycle_coulomb/3, amplitude=1
)

## 2.2 Define the Experiment

In [ ]:
## Create Experiment
exp = Experiment(
    "Pulsed Qubit Spectroscopy",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)


#calculate number of repetitions based on integration time
int_time = 2000e-9    # integration time in [s]
num_rep = int(int_time/len_cycle_coulomb)
print(f"Number of repetitions for {int_time*1e3} ms integration time: {num_rep}")


# define pulse sequence
with exp.acquire_loop_rt(uid=("shots"), count=num_rep):
    with exp.section(uid=("initialize"), length=len_cycle_coulomb/2, alignment=SectionAlignment.RIGHT):
        exp.play(signal="coulomb_1", pulse=coulomb_reset, amplitude=0.5)
        exp.play(signal="coulomb_2", pulse=coulomb_reset, amplitude=0.5)

    with exp.section(uid=("manipulation"), length=len_cycle_coulomb/2, alignment=SectionAlignment.RIGHT):
        exp.play(signal="coulomb_1", pulse=coulomb_manipulate, amplitude=0.75)
        exp.play(signal="coulomb_2", pulse=coulomb_manipulate, amplitude=0.75)

        exp.play(signal="drive", pulse=drive_pulse)

## 2.3 Run the Experiment

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=True)

In [ ]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

In [ ]:
# set experiment calibration and signal map
exp.set_signal_map(map_q0)

In [ ]:
compiled_experiment = session.compile(exp)

In [ ]:
session.run(compiled_experiment);

## 2.4 Plot the Results

In [ ]:
show_pulse_sheet("Qubit Spectroscopy",session.compiled_experiment)

In [ ]:
Plotter.plot(session.results)

# 3 Connect to QCoDeS

In [ ]:
import qcodes as qc
from qcodes.tests.instrument_mocks import DummyInstrument

In [ ]:
# generate dummy instruments
my_magnet = DummyInstrument(name='magnet', gates=['Bx', 'By', 'Bz'])
my_LO = DummyInstrument(name='RF_source', gates=['P', 'f'])

In [ ]:
def set_magnet(session, value):
    my_magnet.Bx.set(value)     # set new value in mT
    time.sleep(0.1)             # settling time
    return my_magnet.Bx.get()       # return new value

def set_frequency(session, value):
    my_LO.f.set(value)          # set new value in MHz
    time.sleep(0.1)             # settling time
    return my_LO.f.get()            # return new value

def my_readout(session, Bvalue, fvalue):
    # example: get datapoint or timetrace from Lock-in
    # here: simple print of current settings
    print(f"Magnet at {Bvalue} mT and frequency at {fvalue} MHz.")
    return random()

## 3.1 Define sweep parameter

In [ ]:
magnet_sweep = LinearSweepParameter(
    uid="Bfield_sweep", start=-400, stop=400, count=9, axis_name="Magnetic field (mT)"
)

frequency_sweep = LinearSweepParameter(
    uid="frequency_sweep", start=0, stop=400, count=5, axis_name="Frequency (MHz)"
)

## 3.2 Define Experiment

In [ ]:
## Create Experiment
exp = Experiment(
    "Pulsed Qubit Spectroscopy",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)


#calculate number of repetitions based on integration time
int_time = 2000e-9    # integration time in [s]
num_rep = int(int_time/len_cycle_coulomb)
print(f"Number of repetitions for {int_time*1e3} ms integration time: {num_rep}")


# define pulse sequence

# outer sweep: step magnetic field
with exp.sweep(uid="Bfield_sweep", parameter=magnet_sweep):
    exp.call(set_magnet, value=magnet_sweep)

    # inner sweep: step frequency of RF source
    with exp.sweep(uid="frequency_sweep", parameter=frequency_sweep):
        exp.call(set_frequency, value=frequency_sweep)

        # innermost part: play pulse sequence
        with exp.acquire_loop_rt(uid=("shots"), count=num_rep):
            with exp.section(uid=("initialize"), length=len_cycle_coulomb/2, alignment=SectionAlignment.RIGHT):
                exp.play(signal="coulomb_1", pulse=coulomb_reset, amplitude=0.5)
                exp.play(signal="coulomb_2", pulse=coulomb_reset, amplitude=0.5)

            with exp.section(uid=("manipulation"), length=len_cycle_coulomb/2, alignment=SectionAlignment.RIGHT):
                exp.play(signal="coulomb_1", pulse=coulomb_manipulate, amplitude=0.75)
                exp.play(signal="coulomb_2", pulse=coulomb_manipulate, amplitude=0.75)

                exp.play(signal="drive", pulse=drive_pulse)
        
        # readout: e.g. get value from Lock-in
        exp.call(my_readout, Bvalue=magnet_sweep, fvalue=frequency_sweep)

## 3.3 Run the Experiment

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=True)

# set experiment calibration and signal map
exp.set_signal_map(map_q0)

# register user functions
session.register_user_function(set_magnet)
session.register_user_function(set_frequency)
session.register_user_function(my_readout)

# compile and run the experiment
session.run_all(exp)

In [ ]:
fig = plt.figure()
datapoints = session.results.user_func_results['my_readout']
datapoints = np.array(datapoints).reshape(magnet_sweep.count, frequency_sweep.count)

plt.imshow(datapoints, extent=[frequency_sweep.start, frequency_sweep.stop, magnet_sweep.start, magnet_sweep.stop])
plt.xlabel(frequency_sweep.axis_name)
plt.ylabel(magnet_sweep.axis_name)
plt.show()